# Leetcode中的算法

- 算法不能瞎想。必须找到一种通用的得出算法的方法。

新手向教程。因为我自己也是个萌新。以前总想着算法和我无关，遇到问题调用库就完事了。然而最近突然决定打算转CS，算法变成了不得不补的东西。

## 11. Container With Most Water

<https://leetcode.com/problems/container-with-most-water/description/>

本质上是找到
$$
    \max\left\{\min\{a_i, a_j\} \cdot (j - i) \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

In [ ]:
class Solution:
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        maximum = 0
        left = 0
        right = len(height) - 1
        possibleMaximum = 0
        while left < right:
            possibleMaximum = (right - left) * min(height[left], height[right])
            maximum = max(maximum, possibleMaximum)
            if height[left] <= height[right]:
                left += 1
            else:
                right -= 1
        return maximum

## 53. Maximum Subarray

<https://leetcode.com/problems/maximum-subarray/description/>

本质上是找到
$$
    \max\left\{\sum_{k = i}^{j} a_k \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

用一个叫做[Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem)的算法。有点数学归纳法的感觉。

以
```
A = -2 1 -3 4 -1 2 1 -5
```
为例，定义$B_i$为`A[0: i+1]`的所有子序列中、所有包含第$i$个元素的子序列的和的最大值。

这个很难看懂。我第一次看也很久才看明白。我们按照严格定义来。首先`A[0: i+1]`的所有“包含第$i$个元素的”子序列可以写成一个集合
$$
\begin{align}
    \left\{A[k, i + 1] \middle| k \in [0, i]\right\}
\end{align}
$$

> 以上面的例子为例，$A[0: 3]$的所有子序列是
```
A[0: 1], A[0: 2], A[0: 3], 
A[1: 2], A[1: 3],
A[2: 3]
```
但是包含“第$i$个元素的”子序列只有
```
A[0: 3], A[1: 3], A[2: 3]
```

$B_i$的定义是
$$
\begin{align}
    B_i = \max\left\{\sum_{m = k}^{i} A_m \middle| k \in [0, i - 1]\right\}
\end{align}
$$

> 以上面的例子为例，$B_3$是
```python
max(sum(A[0: 3]), sum(A[1: 3]), sum(A[2: 3]))
```
而**不是**
```python
max(sum(A[0: 1]), sum(A[0: 2]), sum(A[0: 3]), ..., sum(A[2: 3]))
```

> 以上面的例子为例，可以算出所有的$B_i$
```
A -2  1 -3  4 -1  2  1 -5
B -2  1 -2  4  3  5  6  1
```

最后答案是
$$
    \max\{B_i | i \in [0, n - 1]\}
$$

> 以上面的例子为例，最后答案是6。

In [ ]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maximumSummationSoFar = nums[0]
        possibleMaximumSummation = maximumSummationSoFar
        for i in nums[1: ]:
            maximumSummationSoFar = max(i, i + maximumSummationSoFar)
            possibleMaximumSummation = max(maximumSummationSoFar, possibleMaximumSummation)
        return possibleMaximumSummation

很多人可能会和我一样，第一次错误地认为$B_i$的定义是`A[0: i + 1]`的**所有子序列**的和的最大值，而不是**所以包含了第i个元素的**的子序列的和的最大值。

我后来仔细想了这个问题。对于数学归纳法而言，我们需要这一步推到下一步足够方便。如果按照我们臆想的那个错误定义，我们是很难从`A[0: i+1]`的所有子序列的和的最大值、推到`A[0: i+2]`的所有子序列的和的最大值的。但是我们很容易能从`A[0: i+1]`的所有包含第i个元素的子序列的和的最大值、推导出`A[0: i+2]`的所有包含第i+1个元素的子序列的和的最大值的。他们之间的关系就是一个很简单的
$$
\begin{align}
    B_{i + 1} = \max\{B_{i} + A_{i + 1}, A_{i + 1}\}
\end{align}
$$
关系。

可能有人还是没法理解上面这个式子怎么来的。我再仔细讲一遍。一定要记住，$B_i$一定是`A[0: i+1]`的所有包含了第i个元素的子序列的和的最大值。这是什么意思？意思是$B_i$是一个和，这个和一定含有`A[i]`的成分。否则就不满足定义了。

OK，现在我们有了$B_i$了，怎么从$B_i$推到$B_{i + 1}$呢？同样的，$B_{i + 1}$必须包含`A[i+1]`的成分。这样我们就一个一个地试
```
A[i+1: i+2], A[i: i+2], A[i-1: i+2], ..., A[0: i+2]
```
这时候我们发现了一个问题：我们做了重复劳动了。哪里？你想想你刚才是怎么试出$B_i$的？你也是这样一个一个地试的
```
A[i: i+1], A[i-1: i+1], A[i-2: i+1], ..., A[0: i+1]
```
如果你还是看不出来哪里重复劳动了，我再把$B_{i + 1}$的地方写完整给你看
```
A[i+1: i+2] = 0           + A[i+1]
A[i  : i+2] = A[i  : i+1] + A[i+1]
A[i-1: i+2] = A[i-1: i+1] + A[i+1]
...
A[0  : i+2] = A[0  : i+1] + A[i+1]
              ^^^^^^^^^^^
```
有没有看到重复劳动的地方？我们看到，除了第一项`A[i+1: i+2]`也就是`A[i+1]`之外，其他剩下的子序列都是算$B_i$时用到过的子序列、和`A[i+1]`拼起来的。

## 121. Best Time to Buy and Sell Stock

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock/description/>

本质上是找到
$$
    \max\{a_j - a_i | 0 \leq i \leq j \leq n - 1\}
$$

定义第i天的最大利润是$P_i$。那么这天的利润$P_i$和前面几天的利润有什么关系呢？

- 今天股票价格比昨天低

  那我应该昨天就卖了，所以今天的最大利润和昨天的最大利润一样。$P_i = P_{i - 1}$；
  
- 今天股票价格比昨天高、或者和昨天一样高

  那我应该今天卖。因为今天卖肯定比昨天更赚钱。~~今天的利润比昨天会高$a_i - a_{i - 1}$。~~就是这里错了。如果昨天没有卖怎么办？所以今天的利润不是$P_{i - 1} + a_i - a_{i - 1}$，而是$a_i$减去有史以来见过的最低价$m_i$，即$P_i = a_i - m_i$。所以
$$
    P_i = \max\{P_{i - 1}, a_i - m_i\}
$$

这样答案就是$P_{n - 1}$。

In [1]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        purchasePrice = prices[0]
        profit = 0
        for value in prices[1: ]:
            purchasePrice = min(value, purchasePrice)
            profit = max(profit, value - purchasePrice)
        return profit

In [3]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        profits = [0]
        minimumPriceSoFar = prices[0]
        for i in range(len(prices))[1: ]:
            minimumPriceSoFar = min(prices[i], minimumPriceSoFar)
            profits.append(max(profits[i - 1], prices[i] - minimumPriceSoFar))
        return profits[-1]

## 152. Maximum Product Subarray

<https://leetcode.com/problems/maximum-product-subarray/description/>

本质上是要找到
$$
    \max\left\{\prod_{k = i}^{j} a_k \middle| 0 \leq i \leq j \leq n - 1\right\}
$$

定义到包括了第i个元素的最大积为$P_i$。$P_i$和前面的$P_j (j < i)$有什么关系呢？如果做过Maximum Subarray，可能我们很粗略的得出
$$
    P_i = \max\{P_{i - 1} \cdot a_i, a_i\}
$$
但是这时候会隐隐有一种感觉，感觉符号可能会来搅局。一个负号可能就会让最大积一下子变成最小积。仔细想想，上面的这个粗略的猜测有一个致命的问题，就是遇到负数之后，就会抛弃前面的乘积，只会取最新的这个负数。比如
```
2  3  4 -1
```
遇到-1后，$P_i$会马上变成-1。但是如果-1的后面还有负数
```
2  3  4 -1 -1
```
$P_i$会变成1，可是显然按照定义，这是不对的，$P_i$应该等于24。

可能有一些突然，但是我们必须再定义一个$P'_i$，它是包括了第i个元素的最小积。所以完备的$P_i$的定义是
$$
\begin{align}
    P_i &= \max\{P_{i - 1} \cdot a_i, a_i, P'_{i - 1} \cdot a_i\} \\
    P'_i &= \min\{P_{i - 1} \cdot a_i, a_i, P'_{i - 1} \cdot a_i\}
\end{align}
$$

这里$P_i$的定义和之前的Maximum Subarray非常类似，它是**包括了第i个元素成分的**最大积。

所以答案也是$\max\{P_i\}$。

In [7]:
class Solution:
    def maxProduct(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maximumProducts = [nums[0]]
        minimumProducts = [nums[0]]
        for index, value in enumerate(nums[1: ], 1):
            maximumProducts.append(max(maximumProducts[index - 1] * value, minimumProducts[index - 1] * value, value))
            minimumProducts.append(min(minimumProducts[index - 1] * value, maximumProducts[index - 1] * value, value))
        return max(maximumProducts)

## 198. House Robber

<https://leetcode.com/problems/house-robber/description/>

本质上是要找到
$$
    \max\left\{\sum_{k \in K} a_k \right\}, \quad \text{$K$ is a set such that} \begin{cases}
        K \subseteq \{0, 1, \ldots, n - 1\}, \\
        \forall k_1, k_2 \in K, k_1 \neq k_2, |k_1 - k_2| > 1 \\
    \end{cases}
$$

还是动态规划。定义偷到第i家时，获得的最大利润是$P_i$。那么$P_i$与前面的项的关系是什么呢？是
$$
    P_i = \max\{a_i + P_{i - 2}, P_{i - 1}\}
$$
意思是偷到第i家时，有两种选择
- 要么这家不偷
- 要么偷这家，但是前面一家、也就是第i-1家不能偷，否则就触发警报了

最后答案就是$P_{n - 1}$。

In [2]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        oddSum = 0
        evenSum = 0
        for index, value in enumerate(nums):
            if index % 2 == 0:
                evenSum = max(oddSum, evenSum + value)
            else:
                oddSum = max(evenSum, oddSum + value)
        return max(oddSum, evenSum)

In [ ]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        if len(nums) == 1:
            return nums[0]
        maximumSums = [nums[0], max(nums[0], nums[1])]
        for i in range(len(nums))[2: ]:
            maximumSums.append(max(nums[i] + maximumSums[i - 2], maximumSums[i - 1]))
        return maximumSums[-1]

## 122. Best Time to Buy and Sell Stock II

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock-ii/description/>

本质上是要找到
$$
    \max\left\{\sum_{k=1}^N (a_{s_k} - a_{b_k}) \middle| 0 \leq b_1 < s_1 < b_2 < s_2 < \ldots < b_N < s_N \leq n - 1\right\}
$$

这题和上一题的区别是Best Time to Buy and Sell Stock只能交易一次。这里是可以交易无限次，但是限制是每天只能最多做一次交易，并且手里不能同时有两只股票。也就是说不能第一天买，第二天又买，再都第三天卖掉。

设第i天最大利润是$P_i$，那么$P_{i + 1}$和$P_i$有什么关系？如果第i+1天价格$a_{i + 1}$大于第i天价格$a_i$，那说明昨天不应该卖，应该今天卖掉，这样总利润比昨天高$a_{i + 1} - a_i$；如果今天价格比昨天低或者一样，那么昨天一定要卖掉，今天再入手抄底，这样总利润和昨天一样，不变。
$$
\begin{align}
    P_{i + 1} = \begin{cases}
        P_i + a_{i + 1} - a_i, &\qquad a_{i + 1} > a_i \\
        P_i &\qquad a_{i + 1} \leq a_i \\
    \end{cases}
\end{align}
$$

所以答案是原来数列的一阶差分、再过滤掉所有负数项、之后的和。
$$
    \sum_{i = 0}^{n - 2} \max\{a_{i + 1} - a_i, 0\}
$$

In [12]:
Solution().climbStairs(5)

8

In [26]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        return sum(prices[i + 1] - prices[i] if prices[i + 1] > prices[i] else 0 for i in range(len(prices) - 1))

In [27]:
Solution().maxProfit([1,2,3,4,5])

4

## 3. Longest Substring Without Repeating Characters

<https://leetcode.com/problems/longest-substring-without-repeating-characters/description/>

本质上是要找到
$$
    \max\left\{j - i \middle| \forall k_1, k_2 \in [i, j], k_1 \neq k_2: a_{k_1} \neq a_{k_2}\right\}
$$

定义$S_i$为包含了$a_i$的、0到$a_i$之间最长的无重复字符的子字符串。那么$S_{i + 1}$和$S_i$有什么关系呢？
- 如果$a_{i + 1}$不在$S_i$里，那么$S_{i + 1} = \{S_i, a_{i + 1}\}$
- 如果$S_i$里面已经有$a_{i + 1}$了，$S_i$的定义可以保证$S_i$有且只有一次$a_{i + 1}$。找到它出现的地方，从它后面一个字符开始、把$S_i$切开成两段，取后面一段$S'_i$，这样$S_{i + 1} = \{S'_i, a_{i + 1}\}$

答案就是$\max\{S_i\}$。

In [9]:
class Solution:
    def lengthOfLongestSubstring(self, s):
        """
        :type s: str
        :rtype: int
        """
        if len(s) == 0:
            return 0
        if len(s) == 1:
            return 1
        longestSubstringContainingCurrentCharacter = s[0]
        longestSubstringLength = 1
        for value in s[1: ]:
            if value in longestSubstringContainingCurrentCharacter:
                longestSubstringContainingCurrentCharacter = longestSubstringContainingCurrentCharacter[longestSubstringContainingCurrentCharacter.index(value) + 1: ] + value
                longestSubstringLength = max(longestSubstringLength, len(longestSubstringContainingCurrentCharacter))
            else:
                longestSubstringContainingCurrentCharacter = longestSubstringContainingCurrentCharacter + value
                longestSubstringLength = max(longestSubstringLength, len(longestSubstringContainingCurrentCharacter))
        return longestSubstringLength

In [13]:
Solution().lengthOfLongestSubstring("abcabcbb")

3

### 123. Best Time to Buy and Sell Stock III

<https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/description/>

这题和II的区别是，最多交易2次。

本质上是找到
$$
    \max\{\underbrace{\max\{a_j - a_i + a_l - a_k | 0 \leq i < j < k < l \leq n - 1\}}_{交易2次}, \underbrace{\max\{a_j - a_i | 0 \leq i < j \leq n - 1\}}_{交易1次}, \underbrace{0}_{不交易}\}
$$

In [ ]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        

## 746. Min Cost Climbing Stairs

<https://leetcode.com/problems/min-cost-climbing-stairs/description/>

~~假设到了第$i - 1$级楼梯，此时最小成本是$P_{i - 1}$。那么$P_i$和$P_{i - 1}$有什么关系呢？~~

~~如果第$i - 1$级楼梯是被略过的，那么不能跨过超过2级楼梯，第$i$级楼梯必须上去，所以$P_i = P_{i - 1} + a_i$。~~

~~如果第$i - 1$级楼梯没有被略过，是花了成本的，那么可以比较一下$a_i$和$a_{i - 1}$，如果~~
- ~~发现$a_i$比$a_{i - 1}$小，那说明其实第$i - 1$级刚才应该跳过的，应该直接从第$i - 2$级跳到第$i$级~~
- ~~发现$a_i$比$a_{i - 1}$大，那么还是算了，第$i$级不要上去了吧，观望一下~~
- ~~发现$a_i = a_{i - 1}$，这就有点难处理了。但是根据贪心的原则，还是应该到第$i$级上去。~~

很可惜，上面我想了这么多，最后还是没能做出来。之前也做了这么多道DP题，遇到这种题还是做不出。这里也算是反面教材了一下。事实上DP是很简单的。只要想一个核心问题，就是怎样从前面的条件，比如$i - 1, i - 2$推到$i$，不需要考虑$i + 1$的情况。

怎样到达第$i$级楼梯呢？我们发现只有两种方法
- 从第$i - 2$级跨2级上来
- 从第$i - 1$级跨1级上来

这两种方法里肯定有一种成本最低。那就不管了，直接用一个$\min$套起来
$$
    p_i = \min\{p_{i - 2} + a_{i - 2}, p_{i - 1} + a_{i - 1}\}
$$
这时候千万不要想$i + 1$怎么样，完全不需要想。

注意这个题目里面的$a_i$不是指从到达第$i$级的成本，而是指**从第$i$级出发、上一级或者两级的成本。**所以自然$p_i$也是指的到达第$i$级花的成本，里面是没有$a_i$的成分的。如果你要从第$i$级出发，才需要加上$a_i$。

这时候再去想初始条件。因为一开始可以选择直接跨上第0个台阶，也可以选择直接跨上第1个台阶。**是不需要成本的！**这个条件非常关键，否则你就会写成`p[0] = cost[0], p[1] = min(cost[0], cost[1]）`。

所以完整的递推式是
$$
\left\{\begin{align}
    p_i &= \min\{p_{i - 2} + a_{i - 2}, p_{i - 1} + a_{i - 2}\} \\
    p_0 &= 0 \\
    p_1 &= 0 \\
\end{align}\right.
$$

最终答案不是$p_{n - 1}$而是$p_n$。可以这么理解，最后你要跳到顶层，可以把顶层看成成本为0的一级楼梯。

In [124]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """
        p = [0, 0]
        for index, value in enumerate(cost[2: ] + [0], 2):
            p.append(min(p[index - 2] + cost[index - 2], p[index - 1] + cost[index - 1]))
        print(p)
        return p[-1]

In [125]:
Solution().minCostClimbingStairs([10, 15, 20])

[0, 0, 10, 15]


15

In [126]:
Solution().minCostClimbingStairs([1, 100, 1, 1, 1, 100, 1, 1, 100, 1])

[0, 0, 1, 2, 2, 3, 3, 4, 4, 5, 6]


6

In [127]:
Solution().minCostClimbingStairs([0, 0, 0, 1])

[0, 0, 0, 0, 0]


0

-----

观察上面的几个题目，会发现它们基本上都是这样的形式：找到
$$
    \max\{f(i, j) | 0 \leq i \leq j \leq n - 1\}
$$

- 当$f(i, j) = \sum_{k = i}^{j} a_k$时，变成了Maximum Subarray问题;
- 当$f(i, j) = \min\{a_i, a_j\} \cdot (j - i)$，变成了Container with Most Water问题；
- 当$f(i, j) = a_j - a_i$，变成了Best Time to Buy and Sell Stock问题;
- 当$f(i, j) = \prod_{k = i}^{j} a_k$时，变成了Maximum Product Subarray问题；

## 何时答案是$\max\{P_i\}$、何时答案是$P_{n - 1}$？

## 该怎么选择合适的$P_i$的定义？

## 15. 3Sum

<https://leetcode.com/problems/3sum/description/>

本质上是要找到
$$
    \{(a_i, a_j, a_k) | 0 \leq i < j < k \leq n - 1, a_i + a_j + a_k = 0\}
$$

## 17. Letter Combinations of a Phone Number

<https://leetcode.com/problems/letter-combinations-of-a-phone-number/description/>

In [1]:
import itertools
class Solution:
    def letterCombinations(self, digits):
        """
        :type digits: str
        :rtype: List[str]
        """
        dictionary = {
            "2": list("abc"),
            "3": list("def"),
            "4": list("ghi"),
            "5": list("jkl"),
            "6": list("mno"),
            "7": list("pqrs"),
            "8": list("tuv"),
            "9": list("wxyz")
        }
#         dictionary = dict(zip(list('23456789'),'abc def ghi jkl mno pqrs tuv wxyz'.split()))
#         这样写更好
        return ["".join(i) for i in itertools.product(*[dictionary[j] for j in digits])]

In [2]:
Solution().letterCombinations("23")

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

## 20. Valid Parentheses

<https://leetcode.com/problems/valid-parentheses/description/>

In [16]:
class Solution:
    def isValid(self, s):
        """
        :type s: str
        :rtype: bool
        """
        stack = []
        pairs = {
            "(": ")",
            "[": "]",
            "{": "}"
        }
        for i in s:
            if i in ["(", "[", "{"]:
                stack.append(i)
            else:
                if stack == []:
                    return False
                elif i != pairs[stack[-1]]:
                    return False
                else:
                    stack.pop()
        if stack == []:
            return True
        else:
            return False

In [20]:
Solution().isValid("]")

False

## 12. Integer to Roman

<https://leetcode.com/problems/integer-to-roman/description/>

In [3]:
class Solution:
    def intToRoman(self, num):
        """
        :type num: int
        :rtype: str
        """
        M = ["", "M", "MM", "MMM"] #千
        C = ["", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM"] #百
        X = ["", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC"] #十
        I = ["", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX"] #个
        return M[num//1000] + C[(num%1000)//100] + X[(num%100)//10] + I[num%10]

In [4]:
Solution().intToRoman(1994)

'MCMXCIV'

## 13. Roman to Integer

<https://leetcode.com/problems/roman-to-integer/description/>

## 70. Climbing Stairs

<https://leetcode.com/problems/climbing-stairs/description/>

好像又是动态规划。设$f_n$是n级楼梯的爬法数量，需要找到$f_{n + 1}$和$f_n$的关系。

发现爬到第n+1级有两种办法
- 从第n级爬1级到第n+1级
- 从第n-1级爬2级到第n+1级

可能有人有疑问那么为什么不可以从第n-1级爬1级、再爬1级到第n+1级呢？因为这个方案已经被**从第n级爬1级到第n+1级**包括了。

所以得到了$f_{n + 1}$的表达式
$$
\begin{align}
    f_{n + 1} = f_n + f_{n - 1}
\end{align}
$$
所以可以用动态规划遍历一遍，就得到答案了。

还可以用数学方法降维打击。这个看上去有点像斐波那契数列。但是这个数列从1开始，也就是`1, 2, 3, 5 ...`而不是`1, 1, 2, 3, 5`。

斐波那契数列的通项公式是
$$
    F_n = {\left({1 + \sqrt{5} \over 2}\right)^n - \left({1 - \sqrt{5} \over 2}\right)^n \over \sqrt{5}}
$$
在我们这个题目里面，$f_n = F_{n + 1}$。

为了防止小数运算出错，外面再加一个`round`四舍五入到整数。

数学方法

In [19]:
from math import sqrt
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        return round((((1 + sqrt(5)) / 2) ** (n + 1) - ((1 - sqrt(5)) / 2) ** (n + 1)) / sqrt(5))

In [12]:
Solution().climbStairs(5)

8

遍历做法

In [20]:
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 1
        if n == 2:
            return 2
        q = [0, 1, 2]
        for i in range(3, n + 1):
            q.append(q[i - 1] + q[i - 2])
        return q[-1]

In [25]:
Solution().climbStairs(5)

8

## 171. Excel Sheet Column Number

<https://leetcode.com/problems/excel-sheet-column-number/description/>

In [33]:
class Solution:
    def titleToNumber(self, s):
        """
        :type s: str
        :rtype: int
        """
        return sum((ord(value) - 64) * 26 ** index for index, value in enumerate(s[:: -1]))

In [39]:
Solution().titleToNumber("ZZ")

702

## 168. Excel Sheet Column Title

<https://leetcode.com/problems/excel-sheet-column-title/description/>

171题反过来。设数字是$m$，那么$m$可以表示成
$$
    m = a_{n - 1} \cdot 26^{n - 1} + a_{n - 1} \cdot 26^{n - 2} + \ldots + a_0 \cdot 26^0
$$
其中权重数$a_i$可以这么算出来
$$
\begin{align}
    a_{n - 1} &= m \mod 26^{n - 1} \\
    a_{n - 2} &= (m - a_{n - 1} \cdot 26^{n - 1}) \mod 26^{n - 2} \\
    a_{n - 3} &= (m - a_{n - 1} \cdot 26^{n - 1} - a_{n - 2} \cdot 26^{n - 2}) \mod 26^{n - 3} \\
    \vdots \\
    a_k &= \left(m - \sum_{i = k + 1}^{n - 1} a_k \cdot 26^i\right) \mod 26^k \\
    \vdots \\
    a_0 &= \left(m - \sum_{i = 1}^{n - 1} a_i \cdot 26^i\right) \mod 26^0
\end{align}
$$
但是excel这个计数法则是不正确的，用上面的方法是**错误的，**因为没有0。你想，1是A，26是Z，但是27是AA，而不是A0。A又当0又当1。

```cpp
class Solution {
public:
    string convertToTitle(int n) {
        string ret = "";
        while(n)
        {
            ret = (char)((n-1)%26+'A') + ret;
            n = (n-1)/26;
        }
        return ret;
    }
};
```

In [133]:
class Solution:
    def convertToTitle(self, n):
        """
        :type n: int
        :rtype: str
        """
        s = ""
        while n:
            s = chr((n - 1) % 26 + ord("A")) + s
            n = (n - 1) // 26
        return s

In [134]:
Solution().convertToTitle(702)

'ZZ'

## 190. Reverse Bits

<https://leetcode.com/problems/reverse-bits/description/>

Python内建`bin`做法。

In [136]:
class Solution:
    # @param n, an integer
    # @return an integer
    def reverseBits(self, n):
        return int("{:0>32}".format(bin(n)[2: ])[:: -1], 2)

In [137]:
Solution().reverseBits(43261596)

964176192

另外建一个数，每次左移再加上n的最低位。左移32次。

In [148]:
class Solution:
    # @param n, an integer
    # @return an integer
    def reverseBits(self, n):
        q = 0
        for i in range(32):
            q = (q << 1) + (n & 1)
            n = n >> 1
        return q

In [149]:
Solution().reverseBits(43261596)

964176192

Leetcode显示第一种做法更快。

In [142]:
class Solution(object):
    def hammingWeight(self, n):
        """
        :type n: int
        :rtype: int
        """
        ones = 0
        while n != 0:
            if n % 2:
                ones += 1
            n = n >> 1
        return ones

In [147]:
Solution().hammingWeight(3)

2

## 202. Happy Number

<https://leetcode.com/problems/happy-number/description/>

如果是快乐数，那么一定会出现1。如果不是快乐数，也没有必要一直while循环下去，而是会出现周期。所以需要开一个集合，如果见到1，直接返回True；如果不是1，先看下集合里有没有这个数、也就是看一下以前有没有出现过这个数字，如果出现过了，那就说明开始周期循环了，以后也没有必要再循环下去了，直接返回False。

In [152]:
class Solution(object):
    def isHappy(self, n):
        """
        :type n: int
        :rtype: bool
        """
        numbers = set()
        while True:
            n = sum(int(i) ** 2 for i in str(n))
            if n == 1:
                return True
            
            if n in numbers:
                return False
            else:
                numbers.add(n)

In [153]:
Solution().isHappy(19)

True

## 205. Isomorphic Strings

<https://leetcode.com/problems/isomorphic-strings/description/>

验证A和B之间是否存在一个一一对应的映射。

In [165]:
class Solution(object):
    def isIsomorphic(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        return list(map(s.find, s)) == list(map(t.find, t))

In [166]:
Solution().isIsomorphic("papee", "title")

False

上面的解法本质上是把字符串序列替换成其中每个字符在这个字符串里第一次出现的位置组成的序列，然后比较两个字符串变换后的序列是否相同。这个解法真的很巧妙，我非常喜欢。

为什么验证每个字符第一次出现的位置是否相同和问题是等价的呢?我也没法严格证明这个问题。

In [167]:
class Solution(object):
    def isIsomorphic(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        return len(set(s)) == len(set(t)) == len(set((i, j) for i, j in zip(s, t)))

In [169]:
Solution().isIsomorphic("paper", "title")

True

上面的解法是我想出来的直观的解法。速度稍微慢一点。

前一个等号`len(set(s)) == len(set(t))`保证了两个序列里不会有出现元素种类数目不同。也就是不会出现`abc, abb`这种情况；第二个等号保证了即使元素种类数目相同、同一个也元素不会映射两次，比如`aba, baa`这种情况。

## 219. Contains Duplicate II

<https://leetcode.com/problems/contains-duplicate-ii/description/>

In [5]:
class Solution(object):
    def containsNearbyDuplicate(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: bool
        """
#         if k + 1 > len(nums):
#             k = len(nums) - 1
#         for i in range(len(nums) - k):
#             print(nums[i: i + k + 1])
#             if k + 1 != len(set(nums[i: i + k + 1])):
#                 return True
#         return False
        dictionary = {}
        for index, value in enumerate(nums):
            if value in dictionary: # 如果值已经在里面的话，就判断j - i <= k。
                if index - dictionary[value] <= k:
                    return True # 说明找到了
            dictionary[value] = index # 不管在不在，都需要更新最近坐标
        return False # 找了一圈都没有

In [6]:
Solution().containsNearbyDuplicate([1,2,3,1], 3)

True

In [7]:
Solution().containsNearbyDuplicate([1,0,1,1], 1)

True

In [8]:
Solution().containsNearbyDuplicate([1,2,3,1,2,3], 2)

False

In [9]:
Solution().containsNearbyDuplicate([99, 99], 2)

True

## 263. Ugly Number

<https://leetcode.com/problems/ugly-number/description/>

总之先把所有的数都生成出来存在set里面，再判断就非常非常快了。

In [21]:
import itertools
numbers = {2**i * 3**j * 5**k for i, j, k in itertools.product(range(32), range(21), range(15))}
numbers = {i for i in numbers if i <= 2**31}

class Solution(object):
    def isUgly(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return num in numbers

In [24]:
Solution().isUgly(64)

True

还有一种很巧妙的办法，用数论的方法。因为
$$
    n = 2^i \cdot 3^j \cdot 5^k, \qquad i \in [1, 31], j \in [1, 20], k \in [1, 14]
$$
所以$n$一定能被$2^{31} \cdot 3^{20} \cdot k^{14}$整除，因为
$$
    {2^{31} \cdot 3^{20} \cdot k^{14} \over 2^i \cdot 3^j \cdot 5^k} = 2^{31 - i} \cdot 3^{20 - j} \cdot 5^{14 - k} \in \mathbb{Z}
$$

In [27]:
class Solution(object):
    def isUgly(self, num):
        """
        :type num: int
        :rtype: bool
        """
        #n = (2**31)*(3**20)*(5**14) = 4570198050078720000000000000
        return False if num < 1 or (45701980500787200000000000000) % num != 0 else True

In [33]:
Solution().isUgly(16)

True

这个速度非常快。数学对算法真的是降维打击。

## 342. Power of Four

<https://leetcode.com/problems/power-of-four/description/>

In [57]:
class Solution(object):
    def isPowerOfFour(self, num):
        """
        :type num: int
        :rtype: bool
        """
        return True if 4294967296 % num == 0 else False

In [65]:
Solution().isPowerOfFour(-2147483648)

True

## 290. Word Pattern

<https://leetcode.com/problems/word-pattern/description/>

做这个和上一题[一一对应映射的题](#205.-Isomorphic-Strings)有似曾相识的感觉。

实际上解法一模一样。

In [34]:
class Solution(object):
    def wordPattern(self, pattern, str):
        """
        :type pattern: str
        :type str: str
        :rtype: bool
        """
        str = str.split(" ")
        return list(map(pattern.index, pattern)) == list(map(str.index, str))

In [36]:
Solution().wordPattern("aaaa", "dog cat cat dog")

False

In [54]:
class NumArray(object):

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        if nums == []:
            self.empty = True
            return
        else:
            self.empty = False
        self.nums = nums
        self.integral = [nums[0]]
        for i in nums[1: ]:
            self.integral.append(self.integral[-1] + i)

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        if self.empty == True:
            return 0
        if i == 0:
            return self.integral[j]
        return self.integral[j] - self.integral[i - 1]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# param_1 = obj.sumRange(i,j)

In [55]:
NumArray([-2, 0, 3, -5, 2, -1]).sumRange(2, 5)

-1

In [56]:
NumArray([]).sumRange(2, 5)

0

## 367. Valid Perfect Square

<https://leetcode.com/problems/valid-perfect-square/description/>

In [85]:
class Solution(object):
    def isPerfectSquare(self, num):
        """
        :type num: int
        :rtype: bool
        """
        if num == 0:
            return True
        n = 1
        while n ** 2 <= num:
            if num == n ** 2:
                return True
            n += 1
        return False # 超过了，不再算了

有点看不懂下面这种解法。

In [ ]:
class Solution(object):
    def isPerfectSquare(self, num):
        """
        :type num: int
        :rtype: bool
        """
        start, end = 0, num
        while start < end - 1:
            mid = (start + end)/2
            if mid*mid == num:
                return True
            if mid*mid > num:
                end = mid
            else:
                start = mid
        if start*start == num or end*end == num:
            return True
        return False

In [89]:
# The guess API is already defined for you.
# @param num, your guess
# @return -1 if my number is lower, 1 if my number is higher, otherwise return 0
# def guess(num):

class Solution(object):
    def guessNumber(self, n):
        """
        :type n: int
        :rtype: int
        """
        left = 0
        right = n + 1
        while True:
            currentGuess = int(round((left + right) / 2))
            result = guess(currentGuess)
            if result == 0:
                break
            elif result == 1:
                left = currentGuess
            else:
                right = currentGuess
                
        return currentGuess

In [98]:
Solution().guessNumber(2)

2

## 400. Nth Digit

<https://leetcode.com/problems/nth-digit/description/>

这玩意叫[Champernowne constant](https://en.wikipedia.org/wiki/Champernowne_constant)。

相关资料
- http://oeis.org/A033307
- http://oeis.org/A007376



## 447. Number of Boomerangs

<https://leetcode.com/problems/number-of-boomerangs/description/>

本质上就是找到
$$
    \left\{(a_i, a_j, a_k) \vphantom{\sum}\middle| |a_i - a_j| = |a_i - a_k| \right\}
$$
其中
$$
    a_i, a_j, a_k \in \{a_1, a_2, \ldots, a_n\}, a_i \neq a_j \neq a_k
$$

## 459. Repeated Substring Pattern

<https://leetcode.com/problems/repeated-substring-pattern/description/>

In [4]:
class Solution(object):
    def repeatedSubstringPattern(self, s):
        """
        :type s: str
        :rtype: bool
        """
        for i in range(1, int(len(s) / 2) + 1):
            if len(s) % i == 0:
                if s[0: i] * round(len(s) / i) == s:
                    return True
        return False

In [10]:
Solution().repeatedSubstringPattern("abcabcabca")

False

还有非常非常巧妙的办法。先把字符串$S$复读一遍，变成$S' = \{S, S\}$，再去掉第一个字符、去掉最后一个字符，变成$S''$。再判断$S$是不是$S''$的子字符串。

这个方法我是不可能想出来的，但是我可以给出证明，证明这个方法是能区分复读字符串和非复读字符串的。注意，世界上的字符串要么是复读字符串，要么不是复读字符串，不存在既不是复读字符串也不是非复读字符串的字符串，也不存在既是复读字符串又是非复读字符串的字符串。这一点很重要。所以这个方法的目的是找到一种分割。

充分条件：证明这个方法能识别出复读字符串。证明方法是假设原字符串就是复读字符串，我们需要证明出这个方法能够得出`s in (s + s)[1: -1]`的结论。

假如一个字符串$S$是由一段长度为$p$的字符串重复$n$次得到的（$n > 1$），那么把这个整个字符串再复制一遍，变成$S' = \{S, S\}$，就会是长度为$p$的字符串重复$2 n$次得到的。然后再去掉第一个和最后一个字符，变成$S''$,这样$S''$里面含有$p$重复$2n - 2$次形成的字符串。

因为$n > 1$，所以$2n - 2 > 0$，即$2n - 2 \geq 1$，意思是$S''$里一定至少会出现一次$S$。

所以上面的证明证明了对于复读字符串，这个方法能够识别出来。但是如果一个字符串明明是非复读字符串，这个方法会不会误判为复读字符串呢？接着看下面的必要性证明。

必要条件：证明这个方法还能识别出非复读字符串。证明方法是假设这个方法得出了`s in (s + s)[1: -1]`的结果，证明这个字符串是复读字符串。

假设原字符串$S = \{a_i\} = \{a_1, a_2, \ldots, a_n\}$。复读一遍之后变成了
$$
    S' = \{a_i, a_i\} = \{\underbrace{a_1, a_2, \ldots, a_n}_{S}, \underbrace{a_1, a_2, \ldots, a_n}_{S}\}
$$
再掐头去尾变成了
$$
    S'' = \{a_2, \ldots, a_n, a_1, a_2, \ldots, a_{n- 1}\}
$$
现在存在一个$2 \leq k \leq n$，使得$S''$的一个子字符串
$$
\begin{align}
    &\{a_k, a_{k + 1}, \ldots, a_n, a_1, \ldots, a_{k - 1}\} \\
    = &\{a_1, a_2, \ldots, a_n\}
\end{align}
$$
即（写的好看一点）
$$
\begin{align}
    a_k &= a_1 \\
    a_{k + 1} &= a_2 \\
    a_{k + 2} &= a_3 \\
    \vdots \\
    a_n &= a_{n - k + 1} \\
    a_1 &= a_{n - k + 2} \\
    a_2 &= a_{n - k + 3} \\
    \vdots \\
    a_n &= a_{k - 1}
\end{align}
$$

In [13]:
class Solution:
    def repeatedSubstringPattern(self, s):
        """
        :type s: str
        :rtype: bool
        """
        return s in (s + s)[1: -1]

In [12]:
Solution().repeatedSubstringPattern("abcabc")

True

In [18]:
class Solution:
    def thirdMax(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        nums = set(nums)
        return list(sorted(nums))[-3] if len(nums) >= 3 else max(nums)

In [21]:
Solution().thirdMax([1, 2, 3])

1

In [22]:
class Solution:
    def countSegments(self, s):
        """
        :type s: str
        :rtype: int
        """
        return len(s.split())

In [23]:
Solution().countSegments("Hello, my name is John")

5